In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1441483,2021-07-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1441484,2021-07-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1441485,2021-07-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1441486,2021-07-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
38035,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38037,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38039,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38041,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38043,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
882533,2021-07-07,Arkansas,Arkansas,5001,2259,39.00,5000,Arkansas,AR,Arkansas,State,3017804
882535,2021-07-08,Arkansas,Arkansas,5001,2265,39.00,5000,Arkansas,AR,Arkansas,State,3017804
882537,2021-07-09,Arkansas,Arkansas,5001,2281,39.00,5000,Arkansas,AR,Arkansas,State,3017804
882539,2021-07-10,Arkansas,Arkansas,5001,2281,39.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1441483,2021-07-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1441484,2021-07-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1441485,2021-07-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1441486,2021-07-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-11') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
537,2021-07-11,Snohomish,Washington,40576,611.00,53061,WA,County,822083,4935.75,74.32
1072,2021-07-11,Cook,Illinois,557922,11077.00,17031,IL,County,5150233,10832.95,215.08
1606,2021-07-11,Orange,California,274132,5135.00,6059,CA,County,3175692,8632.20,161.70
2139,2021-07-11,Maricopa,Arizona,564534,10341.00,4013,AZ,County,4485414,12586.00,230.55
2672,2021-07-11,Los Angeles,California,1257185,24542.00,6037,CA,County,10039107,12522.88,244.46
...,...,...,...,...,...,...,...,...,...,...,...
1440523,2021-07-11,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1440795,2021-07-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1441036,2021-07-11,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1441273,2021-07-11,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
537,2021-07-11,Snohomish,Washington,40576,611.00,53061,WA,County,822083,4935.75,74.32,74.32,4935.75
1072,2021-07-11,Cook,Illinois,557922,11077.00,17031,IL,County,5150233,10832.95,215.08,215.08,10832.95
1606,2021-07-11,Orange,California,274132,5135.00,6059,CA,County,3175692,8632.20,161.70,161.70,8632.20
2139,2021-07-11,Maricopa,Arizona,564534,10341.00,4013,AZ,County,4485414,12586.00,230.55,230.55,12586.00
2672,2021-07-11,Los Angeles,California,1257185,24542.00,6037,CA,County,10039107,12522.88,244.46,244.46,12522.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440523,2021-07-11,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1440795,2021-07-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1441036,2021-07-11,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1441273,2021-07-11,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
922958,2021-07-11,Hale,Alabama,2284,77.00,1065,AL,County,14651,15589.38,525.56,525.56,15589.38,1
1004953,2021-07-11,Clarke,Alabama,3547,62.00,1025,AL,County,23622,15015.66,262.47,262.47,15015.66,2
721581,2021-07-11,Lowndes,Alabama,1429,54.00,1085,AL,County,9726,14692.58,555.21,555.21,14692.58,3
639585,2021-07-11,Etowah,Alabama,14297,366.00,1055,AL,County,102268,13979.94,357.88,357.88,13979.94,4
536436,2021-07-11,Franklin,Alabama,4337,81.00,1059,AL,County,31362,13828.84,258.27,258.27,13828.84,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881131,2021-07-11,Sublette,Wyoming,804,7.00,56035,WY,County,9831,8178.21,71.20,71.20,8178.21,19
880660,2021-07-11,Converse,Wyoming,1047,18.00,56009,WY,County,13822,7574.88,130.23,130.23,7574.88,20
1095825,2021-07-11,Lincoln,Wyoming,1446,14.00,56023,WY,County,19830,7291.98,70.60,70.60,7291.98,21
1220381,2021-07-11,Niobrara,Wyoming,163,2.00,56027,WY,County,2356,6918.51,84.89,84.89,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
721581,2021-07-11,Lowndes,Alabama,1429,54.00,1085,AL,County,9726,14692.58,555.21,555.21,14692.58,3,1
922958,2021-07-11,Hale,Alabama,2284,77.00,1065,AL,County,14651,15589.38,525.56,525.56,15589.38,1,2
306238,2021-07-11,Walker,Alabama,7421,285.00,1127,AL,County,63521,11682.75,448.67,448.67,11682.75,30,3
637689,2021-07-11,Clay,Alabama,1609,59.00,1027,AL,County,13235,12157.16,445.79,445.79,12157.16,20,4
778338,2021-07-11,Greene,Alabama,942,35.00,1063,AL,County,8111,11613.86,431.51,431.51,11613.86,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095825,2021-07-11,Lincoln,Wyoming,1446,14.00,56023,WY,County,19830,7291.98,70.60,70.60,7291.98,21,19
776449,2021-07-11,Hot Springs,Wyoming,374,3.00,56017,WY,County,4413,8474.96,67.98,67.98,8474.96,17,20
1068390,2021-07-11,Uinta,Wyoming,2329,13.00,56041,WY,County,20226,11514.88,64.27,64.27,11514.88,5,21
304798,2021-07-11,Teton,Wyoming,3816,11.00,56039,WY,County,23464,16263.21,46.88,46.88,16263.21,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.32,4935.75,20,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.32,4935.75,20,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.32,4935.75,20,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.32,4935.75,20,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.32,4935.75,20,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438123,2021-07-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1438124,2021-07-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1438125,2021-07-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1438126,2021-07-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
919130,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15589.38,2,1,1.00
919131,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15589.38,2,1,0.00
919132,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15589.38,2,1,0.00
919133,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15589.38,2,1,0.00
919134,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15589.38,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114269,2021-07-07,Crook,Wyoming,460,13.00,56011,WY,County,7584,6065.40,171.41,171.41,6091.77,6,23,0.00
1114270,2021-07-08,Crook,Wyoming,461,13.00,56011,WY,County,7584,6078.59,171.41,171.41,6091.77,6,23,1.00
1114271,2021-07-09,Crook,Wyoming,462,13.00,56011,WY,County,7584,6091.77,171.41,171.41,6091.77,6,23,1.00
1114272,2021-07-10,Crook,Wyoming,462,13.00,56011,WY,County,7584,6091.77,171.41,171.41,6091.77,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
718691,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14692.58,1,3,1.00,0.00
718692,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14692.58,1,3,0.00,0.00
718693,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14692.58,1,3,0.00,0.00
718694,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14692.58,1,3,0.00,0.00
718695,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14692.58,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716321,2021-07-07,Albany,Wyoming,4662,17.00,56001,WY,County,38880,11990.74,43.72,43.72,12019.03,23,3,5.00,0.00
716322,2021-07-08,Albany,Wyoming,4670,17.00,56001,WY,County,38880,12011.32,43.72,43.72,12019.03,23,3,8.00,0.00
716323,2021-07-09,Albany,Wyoming,4673,17.00,56001,WY,County,38880,12019.03,43.72,43.72,12019.03,23,3,3.00,0.00
716324,2021-07-10,Albany,Wyoming,4673,17.00,56001,WY,County,38880,12019.03,43.72,43.72,12019.03,23,3,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-11') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
365726,2021-07-11,Imperial,California,28729,740.00,6025,CA,County,181215,15853.54,408.35,408.35,15853.54,1,2,0.00,0.00
2672,2021-07-11,Los Angeles,California,1257185,24542.00,6037,CA,County,10039107,12522.88,244.46,244.46,12522.88,2,5,0.00,0.00
181824,2021-07-11,San Bernardino,California,300636,5117.00,6071,CA,County,2180085,13790.10,234.72,234.72,13790.10,3,4,0.00,0.00
641433,2021-07-11,Inyo,California,1441,38.00,6027,CA,County,18039,7988.25,210.65,210.65,7988.25,4,25,0.00,0.00
81169,2021-07-11,Stanislaus,California,63973,1078.00,6099,CA,County,550660,11617.51,195.77,195.77,11617.51,5,9,0.00,0.00
64545,2021-07-11,San Joaquin,California,75133,1442.00,6077,CA,County,762148,9858.06,189.20,189.20,9858.06,6,17,0.00,0.00
37542,2021-07-11,Riverside,California,303062,4648.00,6065,CA,County,2470546,12267.00,188.14,188.14,12267.00,7,7,0.00,0.00
100686,2021-07-11,Tulare,California,49915,853.00,6107,CA,County,466195,10706.89,182.97,182.97,10706.89,8,10,0.00,0.00
581125,2021-07-11,Merced,California,32479,483.00,6047,CA,County,277680,11696.56,173.94,173.94,11696.56,9,8,0.00,0.00
36558,2021-07-11,Fresno,California,102675,1730.00,6019,CA,County,999101,10276.74,173.16,173.16,10276.74,10,14,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1362643,2021-07-11,Lassen,California,5843,26.00,6035,CA,County,30573,19111.63,85.04,85.04,19111.63,35,1,0.00,0.00
365726,2021-07-11,Imperial,California,28729,740.00,6025,CA,County,181215,15853.54,408.35,408.35,15853.54,1,2,0.00,0.00
778280,2021-07-11,Kings,California,23146,248.00,6031,CA,County,152940,15134.04,162.16,162.16,15134.04,11,3,0.00,0.00
181824,2021-07-11,San Bernardino,California,300636,5117.00,6071,CA,County,2180085,13790.10,234.72,234.72,13790.10,3,4,0.00,0.00
2672,2021-07-11,Los Angeles,California,1257185,24542.00,6037,CA,County,10039107,12522.88,244.46,244.46,12522.88,2,5,0.00,0.00
228717,2021-07-11,Kern,California,111341,1409.00,6029,CA,County,900202,12368.45,156.52,156.52,12368.45,13,6,0.00,0.00
37542,2021-07-11,Riverside,California,303062,4648.00,6065,CA,County,2470546,12267.00,188.14,188.14,12267.00,7,7,0.00,0.00
581125,2021-07-11,Merced,California,32479,483.00,6047,CA,County,277680,11696.56,173.94,173.94,11696.56,9,8,0.00,0.00
81169,2021-07-11,Stanislaus,California,63973,1078.00,6099,CA,County,550660,11617.51,195.77,195.77,11617.51,5,9,0.00,0.00
100686,2021-07-11,Tulare,California,49915,853.00,6107,CA,County,466195,10706.89,182.97,182.97,10706.89,8,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
365726,2021-07-11,Imperial,California,28729,740.00,6025,CA,County,181215,15853.54,408.35,408.35,15853.54,1,2,0.00,0.00
2672,2021-07-11,Los Angeles,California,1257185,24542.00,6037,CA,County,10039107,12522.88,244.46,244.46,12522.88,2,5,0.00,0.00
181824,2021-07-11,San Bernardino,California,300636,5117.00,6071,CA,County,2180085,13790.10,234.72,234.72,13790.10,3,4,0.00,0.00
641433,2021-07-11,Inyo,California,1441,38.00,6027,CA,County,18039,7988.25,210.65,210.65,7988.25,4,25,0.00,0.00
81169,2021-07-11,Stanislaus,California,63973,1078.00,6099,CA,County,550660,11617.51,195.77,195.77,11617.51,5,9,0.00,0.00
64545,2021-07-11,San Joaquin,California,75133,1442.00,6077,CA,County,762148,9858.06,189.20,189.20,9858.06,6,17,0.00,0.00
37542,2021-07-11,Riverside,California,303062,4648.00,6065,CA,County,2470546,12267.00,188.14,188.14,12267.00,7,7,0.00,0.00
100686,2021-07-11,Tulare,California,49915,853.00,6107,CA,County,466195,10706.89,182.97,182.97,10706.89,8,10,0.00,0.00
581125,2021-07-11,Merced,California,32479,483.00,6047,CA,County,277680,11696.56,173.94,173.94,11696.56,9,8,0.00,0.00
36558,2021-07-11,Fresno,California,102675,1730.00,6019,CA,County,999101,10276.74,173.16,173.16,10276.74,10,14,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4802,07/07/21,Lassen,5828,26.00,19062.57,85.04,35,1,6.00,0.00
4803,07/08/21,Lassen,5829,26.00,19065.84,85.04,35,1,1.00,0.00
4804,07/09/21,Lassen,5839,26.00,19098.55,85.04,35,1,10.00,0.00
4805,07/10/21,Lassen,5843,26.00,19111.63,85.04,35,1,4.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4888,07/07/21,Fresno,102616,1727.00,10270.83,172.86,10,14,24.00,0.00
4889,07/08/21,Fresno,102639,1729.00,10273.14,173.06,10,14,23.00,2.00
4890,07/09/21,Fresno,102675,1730.00,10276.74,173.16,10,14,36.00,1.00
4891,07/10/21,Fresno,102675,1730.00,10276.74,173.16,10,14,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)